In [9]:
import pandas as pd
import numpy as np
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import ssl
import tqdm

In [10]:
#header for metadata
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

In [11]:
url1 = "https://housing.com/in/buy/bangalore/bangalore"

In [12]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [13]:
page = requests.get(url1, headers=headers)#, proxies=proxies)
content = page.content
soup = BeautifulSoup(content)
places_list = []
for tag in soup.find_all('a'):
  link = tag.get("href")
  searchd = "/in/buy/bangalore"
  if link.find(searchd) != -1:
    new_link = "https://housing.com" + link
    places_list.append(new_link)
print(places_list)
Total_list=[]
for i in places_list:
  page = requests.get(i, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  link_list = []
  for tag in soup.find_all('a'):
    link = tag.get("href")
    searchd = "/in/buy/projects/page"
    if link.find(searchd) != -1:
      new_link = "https://housing.com" + link
      link_list.append(new_link)
  Total_list.extend(link_list)
print(Total_list)

['https://housing.com/in/buy/bangalore/attibele_1', 'https://housing.com/in/buy/bangalore/devanahalli', 'https://housing.com/in/buy/bangalore/h_cross', 'https://housing.com/in/buy/bangalore/mathikere', 'https://housing.com/in/buy/bangalore/lakshmisagara_1', 'https://housing.com/in/buy/bangalore/chikkagubbi_village', 'https://housing.com/in/buy/bangalore/t_hosahalli', 'https://housing.com/in/buy/bangalore/rayasandra', 'https://housing.com/in/buy/bangalore/carmelaram_bangalore', 'https://housing.com/in/buy/bangalore/sannatammanahalli', 'https://housing.com/in/buy/bangalore/sarjapur', 'https://housing.com/in/buy/bangalore/singasandra', 'https://housing.com/in/buy/bangalore/rmv_extension_stage_2', 'https://housing.com/in/buy/bangalore/yeshwanthpur', 'https://housing.com/in/buy/bangalore/hosakerehalli', 'https://housing.com/in/buy/bangalore/hoodi', 'https://housing.com/in/buy/bangalore/uttarahalli', 'https://housing.com/in/buy/bangalore/mullur', 'https://housing.com/in/buy/bangalore/narayan

In [14]:
def extract_data(url):
  word="Sizes.css-ltuii1::after{content:'\\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}}"
  page = requests.get(url, headers=headers)#, proxies=proxies)
  content = page.content
  soup = BeautifulSoup(content)
  #print(soup)
  result = {}
  flat_data = soup.findAll('div',attrs={'class' : 'css-j2zgcq'})
  flat_name = flat_data[0].find('h1',attrs={'class':'css-1hidc9c'})
  if flat_name is not None:
    flat_name = flat_name.text
  else:
    flat_name = "Flat name not mentioned"
  result['Flat Name'] = flat_name
  flat_address = flat_data[0].find('div',attrs={'class':'css-1ty5xzi'})
  if flat_address is not None:
    flat_address = flat_address.text
  else:
    flat_address = "Flat address not mentioned"
  result['Flat address'] = flat_address
  key = []
  value = []
  word2="Sizes"
  for data in soup.findAll('section',attrs={'class' : 'css-13dph6'}):
    for i in data.findAll('div',attrs={'class' : 'css-0'}):
      if i.text == word:
        key.append(word2)
      else:
        key.append(i.text)
    for j in data.findAll('div',attrs={'class' : 'css-1k19e3'}):
      value.append(j.text)
  for i in range(0,len(value)):
    result[key[i]] = value[i]
  tables = soup.find_all("table")
  table = tables[0]
  key_data=[]
  value_data=[]
  #tab_data = [[cell.text for cell in row.find_all(["th","td"])] for row in table.find_all("tr")]
  for row in table.find_all("tr"):
    for cell in row.find_all("th"):
      key_data.append(cell.text)
    for cell in row.find_all("td"):
      for data in cell.find_all("div"):
        value_data.append(data.text)
  for i in range(0,len(value_data)-1):
    result[key_data[i]] = value_data[i]
  data = soup.findAll("div",attrs={"class":"css-1vyxx8y"})
  for i in data:
    result["About"]=i.text
  images = soup.find_all('img')
  images_list = []
  for image in images: 
    if image['src'].startswith("https://"):
      images_list.append(image['src'])
  result["images"]=images_list
  return result

In [15]:
total_dict=[]
for i in range(0,110):
  print(Total_list[i])
  total_dict.append(extract_data(Total_list[i]))
print(total_dict)

https://housing.com/in/buy/projects/page/254432-gr-samskruthi-by-gr-constructions-in-attibele
https://housing.com/in/buy/projects/page/262144-provident-tivoli-hills-by-provident-housing-limited-in-devanahalli?crf_track=%257B%2522placement%2522%253A%2522pdp_np_fsl%2522%252C%2522adId%2522%253A6279311%257D&fltcnt=b4882556-1317-4d14-bfc6-3c394dc8875e
https://housing.com/in/buy/projects/page/96955-sowparnika-unnathi-by-sowparnika-projects-infrastructure-in-attibele
https://housing.com/in/buy/projects/page/259439-sobha-manhattan-towers-town-park-phase-1-w-4-and-5-by-sobha-limited-in-yadavanahalli
https://housing.com/in/buy/projects/page/257606-right-grishma-by-right-properties-inc-in-attibele
https://housing.com/in/buy/projects/page/611-golden-pearl-by-global-developers-in-attibele
https://housing.com/in/buy/projects/page/103721-om-shri-opulence-by-om-shri-infrastructures-in-attibele
https://housing.com/in/buy/projects/page/259439-sobha-manhattan-towers-town-park-phase-1-w-4-and-5-by-sobha-l

In [16]:
data = pd.DataFrame(total_dict)
data.head(10)

,Flat Name,Flat address,Configurations,Possession Starts,Avg. Price,Sizes,Project Area,Project Size,Launch Date,About,images,Configuration,Possession Status,Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}},Size,Rera Id
0,GR Samskruthi,"No 126, Indlabele Village, Attibele, South Ban...","2, 3 BHK Apartments","Nov, 2025",3.90 K/sq.ft,1075.00 sq.ft. - 1410.00 sq.ft.,4.3 Acres,8 Buildings - 414 units,"Feb, 2021",GR Samskruthi is a project by GR Constructions...,[https://is1-2.housingcdn.com/4f2250e8/0f72862...,NaN,NaN,NaN,NaN,NaN
1,Provident Tivoli Hills,"Devanahalli, Nandi Hill Road, Bangalore",NaN,"Sep, 2023",5.58 K/sq.ft,1200.00 sq.ft. - 3200.00 sq.ft.,61 Acres (51% open),839 units,NaN,Puravankara Tivoli Hills is a residential plot...,[https://is1-2.housingcdn.com/4f2250e8/046397c...,Residential Plots,NaN,NaN,NaN,NaN
2,Sowparnika Unnathi,"Sy No.25/3 A, Indlebele, Attibele, South Banga...","Studio, 1, 2, 3 BHK Studio Apartment, Apartments","Jun, 2022",4.30 K/sq.ft,302.00 sq.ft. - 1068.00 sq.ft.,1 Acres,1 Building - 168 units,"May, 2018",Sowparnika Unnathi is an upcoming housing proj...,[https://is1-2.housingcdn.com/4f2250e8/328787a...,NaN,NaN,NaN,NaN,NaN
3,Sobha Manhattan Towers Town Park Phase 1 W 4 A...,"Phase 1 At Sy.No.48/1A, Yadavanahalli, Attibel...",NaN,"Dec, 2026",8.98 K/sq.ft,1001.00 sq.ft. - 1176.00 sq.ft.,3.8 Acres (75% open),280 units,NaN,Sobha Limited has lauched Sobha Manhattan Towe...,[https://is1-3.housingcdn.com/4f2250e8/dfe68f8...,3 BHK Apartment,NaN,NaN,NaN,NaN
4,Right Grishma,"Site No. 43, 44 and 48, Sreenidhi Sacred Sprin...","1, 2 BHK Apartments","Aug, 2022",3.40 K/sq.ft,695.00 sq.ft. - 1275.00 sq.ft.,0.37 Acres,NaN,NaN,Right Properties Inc appeals to individuals wh...,[https://is1-2.housingcdn.com/4f2250e8/15f227b...,NaN,NaN,NaN,NaN,NaN
5,Golden Pearl,"Sarjapur To Attibele , South Bangalore, Bangalore","3, 4, 5, 6 BHK Villas",NaN,4.57 K/sq.ft,2300.00 sq.ft. - 7000.00 sq.ft.,50 Acres (45% open),600 units,"Dec, 2014",Global Developers has lauched Golden Pearl in ...,[https://is1-3.housingcdn.com/4f2250e8/16edf14...,NaN,Ready to Move,NaN,NaN,NaN
6,Om Shri Opulence,"42/3, Indlabele Road, Near Sarjapur Road, Atti...","2, 3 BHK Apartments","Oct, 2019",3.00 K/sq.ft,870.00 sq.ft. - 1210.00 sq.ft.,1 Acres,2 Buildings - 40 units,"Aug, 2016",There are various amenities in Om Shri Opulenc...,[https://is1-3.housingcdn.com/4f2250e8/1e9c9a6...,NaN,NaN,NaN,NaN,NaN
7,Sobha Manhattan Towers Town Park Phase 1 W 4 A...,"Phase 1 At Sy.No.48/1A, Yadavanahalli, Attibel...",NaN,"Dec, 2026",8.98 K/sq.ft,1001.00 sq.ft. - 1176.00 sq.ft.,3.8 Acres (75% open),280 units,NaN,Sobha Limited has lauched Sobha Manhattan Towe...,[https://is1-3.housingcdn.com/4f2250e8/dfe68f8...,3 BHK Apartment,NaN,NaN,NaN,NaN
8,Om Shri Opulence,"42/3, Indlabele Road, Near Sarjapur Road, Atti...","2, 3 BHK Apartments","Oct, 2019",3.00 K/sq.ft,870.00 sq.ft. - 1210.00 sq.ft.,1 Acres,2 Buildings - 40 units,"Aug, 2016",There are various amenities in Om Shri Opulenc...,[https://is1-3.housingcdn.com/4f2250e8/1e9c9a6...,NaN,NaN,NaN,NaN,NaN
9,Pentagon City,"Attibele, South Bangalore, Bangalore",NaN,NaN,Price on request,1200.00 sq.ft. - 2400.00 sq.ft.,NaN,NaN,NaN,Pentagon City is a residential plot by Reputed...,[https://is1-2.housingcdn.com/4f2250e8/9c76f6e...,Residential Plots,NaN,NaN,NaN,NaN


In [17]:
data["Configurations"]=data['Configurations'].combine_first(data['Configuration'])

In [18]:
data["Sizes"]=data['Sizes'].combine_first(data['Size'])

In [19]:
data=data.rename(columns = {'Possession Starts':'Possession_Starts','Possession Status':'Possession_Status'})

In [20]:
data["Possession_Starts"]=data['Possession_Starts'].combine_first(data['Possession_Status'])

In [21]:
data.head(10)

,Flat Name,Flat address,Configurations,Possession_Starts,Avg. Price,Sizes,Project Area,Project Size,Launch Date,About,images,Configuration,Possession_Status,Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}},Size,Rera Id
0,GR Samskruthi,"No 126, Indlabele Village, Attibele, South Ban...","2, 3 BHK Apartments","Nov, 2025",3.90 K/sq.ft,1075.00 sq.ft. - 1410.00 sq.ft.,4.3 Acres,8 Buildings - 414 units,"Feb, 2021",GR Samskruthi is a project by GR Constructions...,[https://is1-2.housingcdn.com/4f2250e8/0f72862...,NaN,NaN,NaN,NaN,NaN
1,Provident Tivoli Hills,"Devanahalli, Nandi Hill Road, Bangalore",Residential Plots,"Sep, 2023",5.58 K/sq.ft,1200.00 sq.ft. - 3200.00 sq.ft.,61 Acres (51% open),839 units,NaN,Puravankara Tivoli Hills is a residential plot...,[https://is1-2.housingcdn.com/4f2250e8/046397c...,Residential Plots,NaN,NaN,NaN,NaN
2,Sowparnika Unnathi,"Sy No.25/3 A, Indlebele, Attibele, South Banga...","Studio, 1, 2, 3 BHK Studio Apartment, Apartments","Jun, 2022",4.30 K/sq.ft,302.00 sq.ft. - 1068.00 sq.ft.,1 Acres,1 Building - 168 units,"May, 2018",Sowparnika Unnathi is an upcoming housing proj...,[https://is1-2.housingcdn.com/4f2250e8/328787a...,NaN,NaN,NaN,NaN,NaN
3,Sobha Manhattan Towers Town Park Phase 1 W 4 A...,"Phase 1 At Sy.No.48/1A, Yadavanahalli, Attibel...",3 BHK Apartment,"Dec, 2026",8.98 K/sq.ft,1001.00 sq.ft. - 1176.00 sq.ft.,3.8 Acres (75% open),280 units,NaN,Sobha Limited has lauched Sobha Manhattan Towe...,[https://is1-3.housingcdn.com/4f2250e8/dfe68f8...,3 BHK Apartment,NaN,NaN,NaN,NaN
4,Right Grishma,"Site No. 43, 44 and 48, Sreenidhi Sacred Sprin...","1, 2 BHK Apartments","Aug, 2022",3.40 K/sq.ft,695.00 sq.ft. - 1275.00 sq.ft.,0.37 Acres,NaN,NaN,Right Properties Inc appeals to individuals wh...,[https://is1-2.housingcdn.com/4f2250e8/15f227b...,NaN,NaN,NaN,NaN,NaN
5,Golden Pearl,"Sarjapur To Attibele , South Bangalore, Bangalore","3, 4, 5, 6 BHK Villas",Ready to Move,4.57 K/sq.ft,2300.00 sq.ft. - 7000.00 sq.ft.,50 Acres (45% open),600 units,"Dec, 2014",Global Developers has lauched Golden Pearl in ...,[https://is1-3.housingcdn.com/4f2250e8/16edf14...,NaN,Ready to Move,NaN,NaN,NaN
6,Om Shri Opulence,"42/3, Indlabele Road, Near Sarjapur Road, Atti...","2, 3 BHK Apartments","Oct, 2019",3.00 K/sq.ft,870.00 sq.ft. - 1210.00 sq.ft.,1 Acres,2 Buildings - 40 units,"Aug, 2016",There are various amenities in Om Shri Opulenc...,[https://is1-3.housingcdn.com/4f2250e8/1e9c9a6...,NaN,NaN,NaN,NaN,NaN
7,Sobha Manhattan Towers Town Park Phase 1 W 4 A...,"Phase 1 At Sy.No.48/1A, Yadavanahalli, Attibel...",3 BHK Apartment,"Dec, 2026",8.98 K/sq.ft,1001.00 sq.ft. - 1176.00 sq.ft.,3.8 Acres (75% open),280 units,NaN,Sobha Limited has lauched Sobha Manhattan Towe...,[https://is1-3.housingcdn.com/4f2250e8/dfe68f8...,3 BHK Apartment,NaN,NaN,NaN,NaN
8,Om Shri Opulence,"42/3, Indlabele Road, Near Sarjapur Road, Atti...","2, 3 BHK Apartments","Oct, 2019",3.00 K/sq.ft,870.00 sq.ft. - 1210.00 sq.ft.,1 Acres,2 Buildings - 40 units,"Aug, 2016",There are various amenities in Om Shri Opulenc...,[https://is1-3.housingcdn.com/4f2250e8/1e9c9a6...,NaN,NaN,NaN,NaN,NaN
9,Pentagon City,"Attibele, South Bangalore, Bangalore",Residential Plots,NaN,Price on request,1200.00 sq.ft. - 2400.00 sq.ft.,NaN,NaN,NaN,Pentagon City is a residential plot by Reputed...,[https://is1-2.housingcdn.com/4f2250e8/9c76f6e...,Residential Plots,NaN,NaN,NaN,NaN


In [22]:
data = data.drop(columns=['Configuration', 'Possession_Status','Size','Rera Id',"Size.css-ltuii1::after{content:'\e91f';font-family:'housing_phoenix';vertical-align:middle;font-size:14px;margin-left:5px;color:#5e23dc;cursor:pointer;}@media (min-width:1100px){.css-ltuii1::after{font-size:14.5px;}}"])

In [23]:
data.to_csv('Bangalore.csv', index=True)